In [ ]:
import json, os

test_name = 'neuroM_morph_softChecks'
model_results = 'population_models_excerpt-pop/20191218-155115'
val_results_dir='./validation_results'
val_results_path = os.path.join(val_results_dir, test_name, model_results)

if not os.path.exists(val_results_path):
    os.makedirs(val_results_path)
    
pred_file_path = os.path.join(val_results_path,'prediction_summary_FSI_pop.json')
with open(pred_file_path, 'r') as fp:
    pop_prediction_raw = json.load(fp)

In [ ]:
import pandas as pd
prediction_raw = pop_prediction_raw.values()[0]

pred_CellPart_df = dict()
pred_CellPart_df_list = list()
for CellPart in prediction_raw.keys():
    pred_CellPart_df[CellPart] = pd.DataFrame(prediction_raw[CellPart])
    pred_CellPart_df[CellPart].insert(0, 'CellPart', CellPart)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

y_vars=["neurite_X_extent", "neurite_Y_extent", "neurite_Z_extent"]
x_vars=["total_neurite_length", "neurite_field_diameter", "neurite_largest_extent", "neurite_shortest_extent"]

#filepath = os.path.join(val_results_path,'statistics_FSI_pop.pdf')
# data=prediction_raw_df
# sns.pairplot(data, x_vars=x_vars, y_vars=y_vars, hue="NeuriteType", size=4, aspect=1, kind="reg")
# plt.savefig(filepath, dpi=600, )


def corrfunc(x, y, **kws):
    r,p = stats.pearsonr(x,y)
    ax = plt.gca()
    ax.annotate("r = {:.2f}, p ={:.4f}".format(r,p),
                xy=(.1, .9), xycoords=ax.transAxes)

feats_exc = ['total_number_of_neurites', 'CellPart']
for CellPart in prediction_raw.keys():
    prediction_raw_df=pred_CellPart_df[CellPart]
    all_feats=list(prediction_raw_df)
    for feat_name in feats_exc: 
        try:
            all_feats.remove(feat_name)
        except:
            pass
        
    data=prediction_raw_df
    g = sns.PairGrid(data, x_vars=all_feats, y_vars=all_feats, size=5, aspect=1, palette=["red"])
    g.map(sns.regplot)
    g.map(corrfunc)
    plt.subplots_adjust(top=0.95)
    g.fig.suptitle('Cell part: '+CellPart, fontsize=17) 

    # sns.pairplot(data, x_vars=x_vars, y_vars=y_vars, hue="NeuriteType", size=5, aspect=1, kind="reg")
    filepath = os.path.join(val_results_path,'prediction_lreg_'+CellPart+'_FSI_pop.pdf')
    # plt.savefig(filepath, dpi=600, )

In [ ]:
"""    
with sns.axes_style('white'):
    for x,y in zip(x_vars,y_vars):
        sns.jointplot(x, y, data.loc[data.NeuriteType=='axon'], kind='reg')
"""

In [ ]:
for CellPart in prediction_raw.keys():
    prediction_raw_df=pred_CellPart_df[CellPart]
    all_feats=list(prediction_raw_df)
    for feat_name in feats_exc: 
        try:
            all_feats.remove(feat_name)
        except:
            pass
    
    # plt.figure()
    data=prediction_raw_df
    g = sns.pairplot(data, vars=all_feats, diag_kind="kde", kind='reg', markers="+", diag_kws=dict(shade=True))
    g.fig.suptitle('Cell part: '+CellPart, fontsize=17) 

    # plt.show()
    filepath = os.path.join(val_results_path,'prediction_stats_'+CellPart+'_FSI_pop.pdf')
    plt.savefig(filepath, dpi=600, )

In [ ]:
def corrfunc(x, y, **kws):
    r,p = stats.pearsonr(x,y)
    ax = plt.gca()
    ax.annotate("r = {:.2f}, p ={:.2f}".format(r,p),
                xy=(.1, .9), xycoords=ax.transAxes)

feats_exc = ['total_number_of_neurites', 'max_section_branch_order', 'CellPart']
for CellPart in prediction_raw.keys():
    prediction_raw_df=pred_CellPart_df[CellPart]
    all_feats=list(prediction_raw_df)
    for feat_name in feats_exc: 
        try:
            all_feats.remove(feat_name)
        except:
            pass
    
    data_excerpt_df = prediction_raw_df.loc[:, all_feats]
    data=data_excerpt_df
    g = sns.PairGrid(data, palette=["red"])
    g.map_upper(sns.regplot)
    g.map_upper(corrfunc)
    # g.map_upper(plt.reg, s=10)
    g.map_diag(sns.distplot, kde=False)
    g.map_lower(sns.kdeplot, cmap="Blues_d", n_levels=6)
    
    plt.subplots_adjust(top=0.95)
    g.fig.suptitle('Cell part: '+CellPart, fontsize=17) 
    
    filepath = os.path.join(val_results_path,'prediction_stats_'+CellPart+'_FSI_pop.pdf')
    plt.savefig(filepath, dpi=600, )